In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from shapely import Point
pd.set_option('mode.chained_assignment', None)

MAIN_PATH = ''
OUTPUT_DATA_PATH = f'{MAIN_PATH}output_data/'
INPUT_DATA_PATH = f'{MAIN_PATH}input_data/'

# STAGE 1: CLUSTERTING

1. load data – responses, ueqi
2. retrieve all locations (with coordinates) from responses
3. multiply (repeat) the cities that are in the ueqi
4. perform clusterisation by coordinates
5. count cluster centers per cluster
6. if there are clusters with more than 1 ueqi cities in them, divide them
7. if there are clusters with no ueqi cities in them, find their centroid and merge them with the closest cluster
8. get location-clustercenter table
9. replace locations with their respective cluster centers

## 1. load data – responses, ueqi

In [2]:
responses = pd.read_csv(f'{OUTPUT_DATA_PATH}responses.csv')
responses['coordinates_vacancy'] = gpd.GeoSeries.from_wkt(responses['coordinates_vacancy'])
responses['coordinates_cv'] = gpd.GeoSeries.from_wkt(responses['coordinates_cv'])

ueqi = pd.read_csv(f'{OUTPUT_DATA_PATH}ueqi.csv')

## 2. retrieve all locations (with coordinates) from responses

In [3]:
locations = pd.DataFrame(np.concatenate(
    [responses[['address_vacancy','coordinates_vacancy']].drop_duplicates().values,
    responses[['address_cv','coordinates_cv']].drop_duplicates().values]
    ),columns=['address','coordinates']).drop_duplicates(subset='address')

locations['ueqi'] = locations['address'].map(lambda x: x in list(ueqi['address']))

# Print cities present in UEQI but missing in responses:
set(ueqi['address'])-set(locations['address'])

{'Артемовск, Красноярский край', 'Горбатов, Нижегородская область'}

## 3. multiply (repeat) the cities that are in the ueqi

In [4]:
def repeat_rows(df,repeat_mask,repeat_n_times):
    rows_to_repeat = df.query(f'{repeat_mask}==True')
    res = pd.concat(
        [
            df.query(f'{repeat_mask}==False'),
            rows_to_repeat.loc[rows_to_repeat.index.repeat(repeat_n_times)]
            ]).reset_index(drop=True)
    return res

locations_weighted = repeat_rows(locations,'ueqi',2000)

## 4. cluster locations by coordinates

In [5]:
# takes approximately 5 minutes

from sklearn.cluster import KMeans

def get_kmeans(df,n_clusters=100,algorithm='lloyd',clustering_columns=['x','y']):
    X = np.asarray(df[clustering_columns])
    clustering = KMeans(n_clusters=n_clusters,random_state=42,algorithm=algorithm,n_init='auto')
    df['cluster'] = clustering.fit_predict(X)

    return df

locations_weighted['x'] = locations_weighted['coordinates'].map(lambda x: x.x)
locations_weighted['y'] = locations_weighted['coordinates'].map(lambda x: x.y)

locations_clustered = get_kmeans(locations_weighted,n_clusters=len(locations.query('ueqi==True')))

locations_clustered_copy = locations_clustered.copy()
locations_clustered = locations_clustered.drop_duplicates()

## 5. count cluster centers per cluster

In [6]:
cluster_centers_count = locations_clustered.groupby(['cluster','ueqi'])['address'].count().reset_index()
cluster_centers_count = cluster_centers_count.drop_duplicates(subset='cluster',keep='last')

## 6. if there are clusters with more than 1 ueqi cities in them, divide them

In [7]:
def divide_clusters(df, clusters_to_divide):
    df = repeat_rows(df.query('cluster in @clusters_to_divide'),'ueqi',3000)
    res = []
    i_max = df['cluster'].max()

    for i in tqdm(clusters_to_divide):
        #print(i_max)
        cluster = df.query(f'cluster == {i}')
        n_clusters = len(cluster.query('ueqi == True').drop_duplicates())

        new_clusters = get_kmeans(cluster,n_clusters)
        new_cluster_indices = {
            x:y for x,y in zip (
                set(new_clusters['cluster']),np.arange(i_max,i_max+n_clusters))
            }
        i_max = i_max+n_clusters

        new_clusters['cluster'] = new_clusters['cluster'].replace(new_cluster_indices)

        res.append(new_clusters)

    res = pd.concat(res)
    res['cluster'] = res['cluster']+1000

    return res.drop_duplicates()

clusters_to_divide = list(cluster_centers_count.query('ueqi==True and address>1')['cluster'])
new_clusters = divide_clusters(locations_clustered,clusters_to_divide)
locations_clustered = pd.concat([locations_clustered.query('cluster not in @clusters_to_divide'),new_clusters],ignore_index=True)

100%|██████████| 54/54 [00:02<00:00, 21.73it/s]


## 7. if there are clusters with no ueqi cities in them, find their centroid and merge them with the closest cluster

In [8]:
from shapely.geometry import Polygon
from shapely.geometry import Point
from math import radians, cos, sin, asin, sqrt

def haversine(point1, point2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """

    lon1 = point1.x
    lat1 = point1.y
    lon2 = point2.x
    lat2 = point2.y

    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

def merge_clusters(df,clusters_to_merge):
    addresses = list(df.query('ueqi==True')['address'])
    coordinates = [Point(x,y) for x,y in zip(df.query('ueqi==True')['x'],df.query('ueqi==True')['y'])]
    location_coordinates_dict = {x:y for x,y in zip(addresses,coordinates)}

    cluster_centroids = {}
    closest_city = {}
    merge_clusters_dict = {}

    for x in tqdm(clusters_to_merge):
        try:
            cluster_centroids[x] = Polygon(df.query('cluster==@x')['coordinates']).centroid
        except:
            cluster_centroids[x] = df.query('cluster==@x')['coordinates'].iloc[0]

        #print(cluster_centroids[x],x)
        i = np.argmin([haversine(cluster_centroids[x],location_coordinates_dict[y]) for y in location_coordinates_dict])
        closest_city[x] = [*location_coordinates_dict.keys()][i]

    for x in closest_city:
        merge_clusters_dict[x] = df.loc[df['address']==closest_city[x],'cluster'].iloc[0]

    df['cluster'] = df['cluster'].replace(merge_clusters_dict)

    return df

clusters_to_merge = list(cluster_centers_count.query('ueqi==False')['cluster'])
locations_clustered = merge_clusters(locations_clustered,clusters_to_merge)

  0%|          | 0/65 [00:00<?, ?it/s]

100%|██████████| 65/65 [00:02<00:00, 25.42it/s]


## 8. get location-clustercenter table

In [9]:
cluster_centers = locations_clustered.groupby(['cluster','ueqi'])['address'].first().reset_index()
cluster_centers = cluster_centers.query('ueqi==True').drop('ueqi',axis=1)

cluster_centers_dict = {x:y for x,y in zip(cluster_centers['cluster'],cluster_centers['address'])}
location_cluster_dict = {x:y for x,y in zip(locations_clustered['address'],locations_clustered['cluster'])}

## 9. replace locations with their respective cluster centers in the responses table

In [11]:
responses['cluster_cv'] = responses['address_cv'].map(lambda x: location_cluster_dict[x])
responses['cluster_vacancy'] = responses['address_vacancy'].map(lambda x: location_cluster_dict[x])

responses['cluster_center_cv'] = responses['cluster_cv'].map(lambda x: cluster_centers_dict[x])
responses['cluster_center_vacancy'] = responses['cluster_vacancy'].map(lambda x: cluster_centers_dict[x])

In [882]:
responses.to_csv(f'{OUTPUT_DATA_PATH}responses_clustered.csv',index=False)

# STAGE 2: AGGREGATING

1. load data – responses_clustered, specialists, ontology, ueqi
2. group responses by cluster_center_cv and cluster_center_vacancy to get: 
    - distance
    - migrations_total
3. from the groupped table, get all locations and evaluate [to get cities_agg table]:
    - domestic_migrations_count (incoming, when cluster_center_cv==cluster_center_vacancy)
    - incoming_migrations_count (migrations_total - domestic_migrations_count)
    - outcoming_migrations_count (migrations_total - domestic_migrations_count)
4. merge specialists_table with ontology and replace speciality columnn with dummy columns
5. merge responses with the resulting table
6. group responses by cluster_center_vacancy and domains to evaluate the number of opened vacancies per domain in cities
7. merge the resulting groupped table with the groupped table from table 3
8. merge cities_agg with ueqi
9. merge cities_agg with population from city_info
10. compute additional variables:
    - domestic_migrations_count_rel (domestic_migrations_count/population)
    - incoming_migrations_count_rel (incoming_migrations_count/population)
    - outcoming_migrations_count_rel (outcoming_migrations_count/population)
    - migration_ratio (incoming_migrations_count/outcoming_migrations_count)

In [410]:
from scipy.stats import pearsonr,spearmanr,kendalltau

In [435]:
ontology_matching = {'Оператор производственной линии':'Оператор, аппаратчик',
                     'Оператор станков с ЧПУ':'Оператор, аппаратчик',
                     'Монтажник':'Монтажник',
                     'Машинист':'Машинист',
                     'Мастер по ремонту оборудования, техники':'Мастер по ремонту оборудования',
                     'Разнорабочий':'Разнорабочий',
                     'Слесарь, сантехник':'Слесарь',
                     'Геодезист':'Геодезист',
                     'Геолог':'Геолог',
                     'Технолог':'Технолог',
                     'Инженер по охране труда и технике безопасности, инженер-эколог':'Инженер-эколог',
                     'Инженер-конструктор, инженер-проектировщик':'Инженер-конструктор',
                     'Сварщик':'Сварщик', 
                     'Инженер-электроник, инженер-электронщик':'Инженер-проектировщик',
                     'Инженер ПНР':'Наладчик',
                     'Токарь, фрезеровщик, шлифовщик':'Токарь, фрезеровщик, шлифовщик', 
                     'Контролёр ОТК':'Контролер ОТК',
                     #'Инженер-конструктор, инженер-проектировщик':'Инженер-проектировщик', 
                     'Специалист по сертификации':'Специалист по сертификации',
                     'Инженер по качеству':'Инженер по качеству', 
                     'Механик':'Механик',
                     'Инженер по эксплуатации':'Инженер по эксплуатации',
                     'Лаборант':'Лаборант', 
                     'Научный специалист, исследователь':'Исследователь', 
                     'Электромонтажник':'Электромонтажник',
                     'Инженер ПТО, инженер-сметчик':'Инженер-проектировщик',
                     #'Инженер по охране труда и технике безопасности, инженер-эколог':'Инженер по охране труда и технике безопасности',
                    }

In [ ]:
salary_df = salary_df.replace(0,np.nan)

salary_info = salary_df.groupby(['cluster_center_vacancy'])[industries].median().reset_index()
salary_info = salary_info.rename(columns=industries_translation)
salary_info.columns = ['salary_'+x if x != 'cluster_center_vacancy' else x for x in salary_info.columns]

vacancy_count = salary_df.groupby(['cluster_center_vacancy'])[industries].count().reset_index()
vacancy_count = vacancy_count.rename(columns=industries_translation)
vacancy_count.columns = ['vacancy_count_'+x if x != 'cluster_center_vacancy' else x for x in vacancy_count.columns ]

## 1. load data – responses_clustered, specialists, ontology, ueqi, city_info

In [12]:
ontology = pd.read_csv(f'{OUTPUT_DATA_PATH}ontology.csv')
specialists = pd.read_csv(f'{OUTPUT_DATA_PATH}specialists.csv')
city_info = pd.read_csv(f'{OUTPUT_DATA_PATH}city_info.csv')
city_info['address'] = city_info['address'].replace({'Дмитриев-Льговский, Курская область':'Дмитриев, Курская область'})
city_info = city_info.dropna(subset='population')
city_info['address'] = city_info['address'].str.replace('ё','е')
city_info = city_info.drop_duplicates(subset='address')

In [16]:
industries = ['Авиастроение и космическая отрасль', 'Добыча нефти и газа',
              'Добыча угля и металлических руд', 'Машиностроение', 'Судостроение',
              'Фармацевтика', 'Химическая промышленность', 'Цветная металлургия',
              'Черная металлургия', 'Электроника и микроэлектроника']

industries_translation = {'Авиастроение и космическая отрасль':'aircraft_and_space',
                          'Добыча нефти и газа':'oil_and_gas_ext',
                          'Добыча угля и металлических руд':'coal_and_metal_ext',
                          'Машиностроение':'machinery',
                          'Судостроение':'shipbuilding',
                          'Фармацевтика':'pharmacy',
                          'Химическая промышленность':'chemicals',
                          'Цветная металлургия':'nonferrous_metallurgy',
                          'Черная металлургия':'ferrous_metallurgy',
                          'Электроника и микроэлектроника':'electronics'}

## 2. group responses by cluster_center_cv and cluster_center_vacancy to get distance and migrations_total

In [13]:
migrations = responses.groupby(['cluster_center_cv','cluster_center_vacancy'])['id_response'].count().rename('migrations_total').reset_index()

migrations_temp = responses.groupby(['cluster_center_cv','cluster_center_vacancy']).agg(
    {'coordinates_cv':'first','coordinates_vacancy':'first'}).reset_index()
#migrations_temp['distance'] = migrations_temp.apply(lambda x:haversine(x.coordinates_cv,x.coordinates_vacancy),axis=1)

migrations = migrations.merge(migrations_temp[['cluster_center_cv','cluster_center_vacancy','distance']])

## 3. from the groupped table, get all locations and evaluate domestic_migrations_count, incoming_migrations_count, and outcoming_migrations_count

In [14]:
cities_agg = migrations.query('cluster_center_cv==cluster_center_vacancy').drop_duplicates(
    subset=['cluster_center_vacancy'])[['cluster_center_cv','migrations_total']]

a = set(list(migrations['cluster_center_cv'])+list(migrations['cluster_center_vacancy']))
b = set(cities_agg['cluster_center_cv'])
cities_agg = pd.DataFrame(np.concatenate([cities_agg.values,[[x,0] for x in list(a-b)]]),columns=['city','migrations_domestic'])
cities_agg['migrations_domestic']= cities_agg['migrations_domestic'].astype(int)

cities_agg = cities_agg.merge(migrations.groupby('cluster_center_vacancy')['migrations_total'].sum().reset_index().rename(
    columns={'cluster_center_vacancy':'city','migrations_total':'migrations_incoming'}),how='left').fillna(0)
cities_agg['migrations_incoming'] = cities_agg['migrations_incoming']-cities_agg['migrations_domestic']
cities_agg['migrations_incoming'] = cities_agg['migrations_incoming'].astype(int)

cities_agg = cities_agg.merge(migrations.groupby('cluster_center_cv')['migrations_total'].sum().reset_index().rename(
    columns={'cluster_center_cv':'city','migrations_total':'migrations_outcoming'}),how='left').fillna(0)
cities_agg['migrations_outcoming'] = cities_agg['migrations_outcoming']-cities_agg['migrations_domestic']
cities_agg['migrations_outcoming'] = cities_agg['migrations_outcoming'].astype(int)

## 4. merge specialists with ontology and replace speciality columnn with dummies

In [17]:
specialists = specialists.merge(ontology[['speciality','prof_domain']],how='left')
specialists = pd.concat([specialists,pd.get_dummies(specialists['prof_domain'])],axis=1).drop(
    'prof_domain',axis=1).groupby(['id_candidate','id_cv','speciality']).max().reset_index()
#specialists = specialists.rename(columns=industries_translation)

## 5. merge responses with the resulting table

In [53]:
responses_domains = responses.merge(specialists,how='left')

## 6. group responses by cluster_center_vacancy and domains to evaluate the number of opened vacancies per domain in cities

In [54]:
ontology['prof_domain'] = ontology['prof_domain'].replace(industries_translation)

In [55]:
domains = list(set(ontology['prof_domain']))
city_responses_per_domain = responses_domains.groupby('cluster_center_vacancy').agg({x:'sum' for x in domains}).replace(
    {True:1,False:0}).reset_index().rename(columns={'cluster_center_vacancy':'city'})

In [56]:
city_responses_per_domain.columns = ['responses_'+x if x not in 'city' else x for x in city_responses_per_domain.columns]

## 7. merge the resulting groupped table with the cities_agg table

In [212]:
cities_agg = cities_agg.merge(city_responses_per_domain,how='left')

for x in city_responses_per_domain.drop('city',axis=1).columns:
    cities_agg[x] = cities_agg[x].fillna(0).astype(int)

## 8. merge cities_agg with ueqi

In [67]:
cities_agg = cities_agg.merge(ueqi.drop(['city','region'],axis=1),left_on='city',right_on='address',how='left').drop_duplicates(
    subset=['city']).drop('address',axis=1)

In [70]:
cities_agg = cities_agg.rename(columns={'greens_spaces':'green_spaces'})
cities_agg = cities_agg.rename(columns={'residential':'ueqi_residential',
                           'street_networks':'ueqi_street_networks',
                           'green_spaces':'ueqi_green_spaces',
                           'public_and_business_infrastructure':'ueqi_public_and_business_infrastructure',
                           'social_and_leisure_infrastructure':'ueqi_social_and_leisure_infrastructure',
                           'citywide_space':'ueqi_citywide_space'})

## 9. merge cities_agg with population from city_info

In [71]:
cities_agg = cities_agg.merge(city_info[['address','population']],how='left',left_on='city',right_on='address').drop('address',axis=1)
cities_agg['population'] = cities_agg['population'].astype(int)
cities_agg = cities_agg.merge(locations.drop('ueqi',axis=1),how='left',left_on='city',right_on='address')
cities_agg = cities_agg.drop('address',axis=1)

## 10. compute additional variables

In [74]:
def get_migration_coeff(incoming_migrations,outcoming_migrations):
    if outcoming_migrations > incoming_migrations:
        try:
            res = -(outcoming_migrations/incoming_migrations-1)
        except:
            res = -np.inf
    else:
        try:
            res = incoming_migrations/outcoming_migrations-1
        except:
            res = np.inf

    return res

cities_agg['migrations_rel_domestic'] = cities_agg['migrations_domestic']/cities_agg['population']
cities_agg['migrations_rel_incoming'] = cities_agg['migrations_incoming']/cities_agg['population']
cities_agg['migrations_rel_outcoming'] = cities_agg['migrations_outcoming']/cities_agg['population']
cities_agg['migration_coeff'] = [get_migration_coeff(x,y) for x,y in zip(
    cities_agg['migrations_incoming'],
    cities_agg['migrations_outcoming'])]

In [289]:
cities_agg.to_csv(f'{OUTPUT_DATA_PATH}cities_aggregated.csv')